# Lab4.1: HelloSparkSQL

## Tổng quan bài tập
**Đề bài**: Dựa vào tập dữ liệu, hãy tính số lượng người có tuổi (`Age`) dưới 40 theo từng đất nước (`Country`) dựa vào cơ chế chạy truy vấn SQL trong Spark. Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán trên.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1E0dmuhCV0Rfg6EoWrryrvE0aZVfM-jCS/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .csv với cấu trúc như sau:
```
root
 |-- Timestamp: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- self_employed: string (nullable = true)
 |-- family_history: string (nullable = true)
 |-- treatment: string (nullable = true)
 |-- work_interfere: string (nullable = true)
 |-- no_employees: string (nullable = true)
 |-- remote_work: string (nullable = true)
 |-- tech_company: string (nullable = true)
 |-- benefits: string (nullable = true)
 |-- care_options: string (nullable = true)
 |-- wellness_program: string (nullable = true)
 |-- seek_help: string (nullable = true)
 |-- anonymity: string (nullable = true)
 |-- leave: string (nullable = true)
 |-- mental_health_consequence: string (nullable = true)
 |-- phys_health_consequence: string (nullable = true)
 |-- coworkers: string (nullable = true)
 |-- supervisor: string (nullable = true)
 |-- mental_health_interview: string (nullable = true)
 |-- phys_health_interview: string (nullable = true)
 |-- mental_vs_physical: string (nullable = true)
 |-- obs_consequence: string (nullable = true)
 |-- comments: string (nullable = true)
```


# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,046 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-secu

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# HelloSparkSQL
Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('Lab4.1')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/My Drive/DEP303/sample.csv'

In [ ]:
surveyDF = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(DATASET_PATH)

# Create temp view
surveyDF.createOrReplaceTempView('survey_tbl')
# Run query
countDF = spark.sql("""select Country, count(1) as count
  from survey_tbl
  where Age<40
  group by Country""")

countDF.show()

+--------------+-----+
|       Country|count|
+--------------+-----+
| United States|    4|
|        Canada|    2|
|United Kingdom|    1|
+--------------+-----+

